# Hand in 7

## Publisher

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import ast

In [47]:
data = pd.read_csv("diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [49]:
data = data.drop(columns=["Pregnancies","SkinThickness","BMI","DiabetesPedigreeFunction","Age"])
data

,Glucose,BloodPressure,Insulin,Outcome
0,148,72,0,1
1,85,66,0,0
2,183,64,0,1
3,89,66,94,0
4,137,40,168,1
...,...,...,...,...
763,101,76,180,0
764,122,70,0,0
765,121,72,112,0
766,126,60,0,1


In [51]:
# Our data seem to have a lot of spread between the different columns to avoid a feature having more weigth we need to scale the data
scaler = MinMaxScaler()
scaler

MinMaxScaler()

In [53]:
# We scale the three coloumns mentioned before, making sure not to scale the label
scaled_data = scaler.fit_transform(data[["Glucose","BloodPressure","Insulin"]])
scaled_data

array([[0.74371859, 0.59016393, 0.        ],
       [0.42713568, 0.54098361, 0.        ],
       [0.91959799, 0.52459016, 0.        ],
       ...,
       [0.6080402 , 0.59016393, 0.13238771],
       [0.63316583, 0.49180328, 0.        ],
       [0.46733668, 0.57377049, 0.        ]])

In [55]:
data[["Glucose","BloodPressure","Insulin"]] = scaled_data
data

,Glucose,BloodPressure,Insulin,Outcome
0,0.743719,0.590164,0.000000,1
1,0.427136,0.540984,0.000000,0
2,0.919598,0.524590,0.000000,1
3,0.447236,0.540984,0.111111,0
4,0.688442,0.327869,0.198582,1
...,...,...,...,...
763,0.507538,0.622951,0.212766,0
764,0.613065,0.573770,0.000000,0
765,0.608040,0.590164,0.132388,0
766,0.633166,0.491803,0.000000,1


In [7]:
import time
import paho.mqtt.client as mqtt

# We create a client as the data publisher
mqttc = mqtt.Client()
# We are connecting to port 1883 and set *keepalive* parameter to 60; *keepalive* is the maximum waiting time in seconds.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.

length = len(data)

if __name__ == "__main__":
    print("Publishing...")
    
    index = 0
    while (index >= 0):
        # Get the current data reading to send out
        S = data.iloc[index % length]
        
        # Publish the data reading as 'diabetes_data'
        #Publisher is open to all! Make sure use your own ‘Data name’ in your exercise of Publisher/Subscriber
        mqttc.publish("diabetes_data", str(
        f'{{"Glucose": {S["Glucose"]}, "BloodPressure": {S["BloodPressure"]}, "Insulin": {S["Insulin"]},"Outcome": {S["Outcome"]}}}'
                                          )
                     )
        
        # We send the next reading after 2 seconds
        time.sleep(1)
        index = index + 1
       # if index == length+1:
        #    index = 0
         #   print("-----------------------")
          #  print("restarting from index 0")
           # print("-----------------------")

Publishing...


KeyboardInterrupt: 

In [13]:
dict1 = f'{{"Glucose": {S["Glucose"]}, "BloodPressure": {S["BloodPressure"]}, "Insulin": {S["Insulin"]},"Outcome": {S["Outcome"]}}}'

In [15]:
dict1

'{"Glucose": 0.5577889447236181, "BloodPressure": 0.5081967213114754, "Insulin": 0.21513002364066194,"Outcome": 0.0}'

In [17]:
data_test = ast.literal_eval(dict1)
data_test

{'Glucose': 0.5577889447236181,
 'BloodPressure': 0.5081967213114754,
 'Insulin': 0.21513002364066194,
 'Outcome': 0.0}

In [19]:
test = pd.DataFrame(data_test, index = [1])
test

,Glucose,BloodPressure,Insulin,Outcome
1,0.557789,0.508197,0.21513,0.0


In [71]:
data["Outcome"].values[299]


0

In [61]:
dummy = pd.get_dummies(data["Outcome"])
dummy

,0,1
0,False,True
1,True,False
2,False,True
3,True,False
4,False,True
...,...,...
763,True,False
764,True,False
765,True,False
766,False,True
